
# Turn Your AI Into GBA Code!

Ever dreamed of running your own neural network on a real Game Boy Advance? That’s exactly what this project lets you do—convert your ONNX model into C code that runs right inside Pokémon Emerald!

Last time, we trained a model. Now, let’s bring it to life on the GBA.

## What You’ll Do in This Tutorial 
- Load your trained ONNX model
- Quantize it to full int8 for retro hardware
- Run and test it directly in the emulator

### Imports

In [ ]:
import sys
sys.path.append("..")

import os

import numpy as np
import torch
import torch.nn as nn
import onnx
import onnxruntime as ort
from onnx import shape_inference
from onnx import numpy_helper

import torch.nn.functional as F

from pkmn_rl_arena.quantize.quantize import FullQuantizer
from pkmn_rl_arena.export.onnx_exporter import ONNXExporter
from pkmn_rl_arena.paths import PATHS

from pkmn_rl_arena.export.passes.delete_pass import (
    DeleteFirstInputQDQPass,
    DeleteQuantizePass,
    DeleteFirstLastQuantizeDequantizePass,
)
from pkmn_rl_arena.export.passes.fusion_pass import (
    GemmQuantDequantFusionPass,
)

from pkmn_rl_arena.export.base import ExportBaseGba
from pkmn_rl_arena.export.onnx_utils import OnnxUtils
from pkmn_rl_arena.data.parser import MapAnalyzer

from pkmn_rl_arena.export.exporters.parameters import ExportParameters
import rustboyadvance_py



### Set Model Paths

Before moving forward, let's organize the three key ONNX files we'll use:

- **Original ONNX:** The model you trained, in full precision.
- **Quantized ONNX:** Converted to int8 for hardware compatibility, and used to test inference with ONNX Runtime.
- **Fused ONNX:** Created from the quantized model, with Gemm and QDQ operations fused together so our export module can process it for the GBA.

Defining these paths now will make the next steps much smoother.

In [ ]:
onnx_path = "pokemon_battle_model.onnx"
quantized_onnx_path = "pokemon_battle_model_quantized.onnx"
fused_path = "pokemon_battle_model_quantized_fused.onnx"

You can print the actual graph with ONNX to see the changes we make.  
For a clearer visualization, try Netron—a handy tool for exploring ONNX model graphs.

In [ ]:
onnx_raw = onnx.load(onnx_path)
print(onnx.helper.printable_graph(onnx_raw.graph))

## Quantize the model

Quantization is the process of converting your model’s weights and activations from floating point to int8, making it much more efficient for hardware like the GBA.  

To do this, we need to calibrate QDQ (Quantize-Dequantize) pairs using sample data, which helps the model learn how to scale values correctly.

> Note:  
> Calibration works best with real data from your application. In future versions, we’ll provide a tool to help generate calibration data automatically.

In [ ]:

num_samples = 10

quantizer = FullQuantizer(onnx_path, quantized_onnx_path)
calib_reader = FullQuantizer.create_fake_calibration_data(
    onnx_path, num_samples=num_samples
)
quantizer.quantize(calib_reader)

# Infer shapes
quantized_model = onnx.load(quantized_onnx_path)
inferred_model = shape_inference.infer_shapes(quantized_model)
onnx.save(inferred_model, quantized_onnx_path)

print(onnx.helper.printable_graph(inferred_model.graph))

### Get the GBA template folder

To test your exported model, a ready-to-use GBA project folder is provided.  
This folder contains everything needed to build and run your model on the emulator.

(and also on the real hardare !)

In [ ]:
ExportBaseGba.copy_gba_folder(".")

### Generating random inputs

To test our exported model, we generate random inputs and run inference using both ONNX Runtime and the exported GBA model.

Since the exported model uses full int8 and the ONNX model uses QDQ pairs, their input and output formats differ.  
To compare results accurately, we need to retrieve the input and output scaling factors and use them to convert values between formats.

In [ ]:
quantized_model = onnx.load(quantized_onnx_path)
output_scale = OnnxUtils.get_last_qdq_scaling_factor(quantized_model.graph)[0]
input_scale = OnnxUtils.get_first_qdq_scaling_factor(quantized_model.graph)[0]

input_random = np.random.uniform(-1, 1, (1, 360)).astype(np.float32)

Next, we run inference with ONNX Runtime to get the model outputs.  
This lets us compare the results from the quantized ONNX model with those from the exported GBA model.

In [ ]:

ort_session = ort.InferenceSession(quantized_onnx_path)
input_type = ort_session.get_inputs()[0].type
if 'int8' in input_type:
    model_input = input_random
else:
    model_input = input_random.astype(np.float32)
ort_outputs = ort_session.run(None, {ort_session.get_inputs()[0].name: model_input})
onnx_output = ort_outputs[0]
print("ONNX Runtime output:", onnx_output)

## Fuse and delete nodes

To make the model compatible with our export module, we need to fuse QDQ pairs and Gemm operations.

This process creates a custom QGemm node, simplifying the graph and making it easier to generate the corresponding C code for the GBA.

In [ ]:
fusion_pass = GemmQuantDequantFusionPass()
delete_pass = DeleteQuantizePass()

fusion_pass.run(quantized_model.graph)
delete_pass.run(quantized_model.graph)
onnx.save(quantized_model, fused_path)

fused_model = onnx.load(fused_path)
print(onnx.helper.printable_graph(fused_model.graph))

## Generate the export

To generate the export, simply call the export function.  
This will create the necessary header files containing all model parameters (weights and biases), and a `forward.c` file that implements the model’s inference logic for the GBA.

In [ ]:
exporter = ONNXExporter(fused_path)
exporter.export(output_dir="gba")

### Compile it !

In [ ]:
os.system("make -C gba")

## Communicate with the rust emulator

To run the emulator, we need to set up stop addresses, write the input data, and read the output.  
For a detailed explanation of how this works, check out our tutorial:

https://github.com/wissammm/PkmnRLArena/wiki/How-stopHandleTurn-works

```c
volatile u16 stopWriteData IN_EWRAM;
volatile u16 stopReadData IN_EWRAM;
volatile int8_t input[1024] IN_EWRAM;
volatile int8_t output[10] IN_EWRAM;

int main(void)
{
    ...
    stopWriteData = 1;
    forward(input, output);
    stopReadData = 1;
    ...
}
```

In [ ]:
def setup_gba_environment(rom_path, map_path):
    """Setup GBA environment and return necessary objects."""
    gba = rustboyadvance_py.RustGba()
    gba.load(PATHS["BIOS"], rom_path)
    parser = MapAnalyzer(map_path)
    addr_write = int(parser.get_address("stopWriteData"), 16)
    addr_read = int(parser.get_address("stopReadData"), 16)
    gba.add_stop_addr(addr_write, 1, True, "stopWriteData", 3)
    gba.add_stop_addr(addr_read, 1, True, "stopReadData", 4)

    output_addr = int(parser.get_address("output"), 16)
    input_addr = int(parser.get_address("input"), 16)

    return gba, parser, addr_write, addr_read, output_addr, input_addr


def run_gba_inference(
    gba, addr_write, input_addr, output_addr, input_data, output_size
):
    """Run inference on GBA and return results."""
    # Wait for initial stop
    id = gba.run_to_next_stop(20000)
    while id != 3:
        id = gba.run_to_next_stop(20000)

    # Write input data
    gba.write_i8_list(input_addr, input_data.reshape(-1).tolist())
    gba.write_u16(addr_write, 0)

    # Wait for computation to complete
    id = gba.run_to_next_stop(20000)
    while id != 4:
        id = gba.run_to_next_stop(20000)

    # Read output
    output_read = gba.read_i8_list(output_addr, output_size)
    return np.array(output_read, dtype=np.int8).reshape(-1)


In [ ]:

gba, parser, addr_write, addr_read, output_addr, input_addr = (
    setup_gba_environment("gba/gba.elf", "gba/build/gba.map")
)
# Scale and convert input to int8
input_gba = np.round(input_random / input_scale).astype(np.int8)
gba_output = run_gba_inference(gba, addr_write, input_addr, output_addr, 
                            input_gba, 10)


## Compare Results

Finally, we compare the outputs from ONNX Runtime and the GBA model. 

Since both outputs are in different formats, we dequantize them using the scaling factors to bring them to the same range.  
This allows us to check if the results match (within a reasonable tolerance).

In [ ]:
onnx_float = onnx_output.astype(np.float32).reshape(-1)
gba_float = (gba_output.astype(np.float32)) * output_scale

print(f"ONNX output (float):{onnx_float}")
print(f"GBA output (float):    {gba_float}")
float_match = np.allclose(onnx_float, gba_float, rtol=1e-1, atol=6e4)
if float_match:
    print("Dequantized outputs match with (a lot of) tolerance!")
else:
    print("Outputs do not match :O")
